In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

<h1> important!</h1>

install these packages:

pip install beautifulsoup4 <br>
pip install requests<br>
pip install lxml

<h1>Request library</h1>
<p> in here we are just sending our request to our desired website then we can load our html in our r object</p>

In [11]:
url = 'https://www.scrapethissite.com/pages/simple/' #visit the website first

r=requests.get(url)
r

<Response [200]>

<h1>Beautiful soup</h1>
<p> with beautiful soup and xlib we can parse our html doc and search for different tags and attributes.</p>

<h2> example 1 </h2>

In [12]:
soup=BeautifulSoup(r.content, 'html.parser')
country_name = []
country_capital = []
country_pop = []
country_area = []
# soup.find_all('section', 'countries')
countries=soup.find('section').find_all('div', 'col-md-4 country')
countries[0].find_all('span')
for country in countries:
    country_name.append(str(country.find('h3').text.strip()))
    country_capital.append(str(country.find('span','country-capital').text))
    country_pop.append(int(country.find('span','country-population').text))
    country_area.append(float(country.find('span','country-area').text))

<h2>optimize version!</h2>

In [13]:
countries = soup.select('section div.col-md-4.country')
country_name = [c.find('h3').text.strip() for c in countries]
country_capital = [c.find('span', 'country-capital').text for c in countries]
country_pop = [int(c.find('span', 'country-population').text) for c in countries]
country_area = [float(c.find('span', 'country-area').text) for c in countries]


<h2>adding to df</h2>

In [14]:
import pandas as pd


df = pd.DataFrame(data= {'country': country_name, 'capital': country_capital, 'population': country_pop, 'area':country_area})
df

,country,capital,population,area
0,Andorra,Andorra la Vella,84000,468.0
1,United Arab Emirates,Abu Dhabi,4975593,82880.0
2,Afghanistan,Kabul,29121286,647500.0
3,Antigua and Barbuda,St. John's,86754,443.0
4,Anguilla,The Valley,13254,102.0
...,...,...,...,...
245,Yemen,Sanaa,23495361,527970.0
246,Mayotte,Mamoudzou,159042,374.0
247,South Africa,Pretoria,49000000,1219912.0
248,Zambia,Lusaka,13460305,752614.0


<h2> example two</h2>

<b>link = https://www.scrapethissite.com/pages/forms/?page_num=1&per_page=100 vist this link first </b>

this webpage has diffrent pages that each containing 100 row of data.

we grab all those data and store them in our df.

In [ ]:
team_name = []
year = []
wins = []
losses = []
pct = []
gf= []
ga= []
diff =[]

In [10]:
for i in tqdm(range(1, 7)):
    r = requests.get(f'https://www.scrapethissite.com/pages/forms/?page_num={i}&per_page=100')
    soup  = BeautifulSoup(r.content, 'html.parser')
    child_table=soup.find('table')
    newSoup=child_table.find_all('tr', class_='team')
    team_name.extend([str(c.find('td', class_= 'name').text.strip()) for c in newSoup])
    year.extend([int(c.find('td', class_= 'year').text.strip()) for c in newSoup])
    wins.extend([int(c.find('td', class_= 'wins').text.strip()) for c in newSoup])
    losses.extend([int(c.find('td', class_= 'losses').text.strip()) for c in newSoup])
    # ot_losses = [int(c.find('td', class_= 'ot-losses').text.strip()) if isinstance(c.find('td', class_= 'ot-losses').isdigit(), int)== True  else 'None'  for c in newSoup ]
    pct.extend([float(c.find('td', class_= 'pct').text.strip()) for c in newSoup])
    gf.extend([int(c.find('td', class_= 'gf').text.strip()) for c in newSoup])
    ga.extend([int(c.find('td', class_= 'ga').text.strip()) for c in newSoup])
    diff.extend([int(c.find('td', class_= 'diff').text.strip()) for c in newSoup])


100%|██████████| 6/6 [00:17<00:00,  2.93s/it]


In [7]:
len(team_name)

582

In [8]:
rows = list(zip(team_name, year, wins, losses, pct, gf, ga, diff))
columns = ['Team Name Year', 'Wins', 'Losses', 'OT Losses', 
           'Win %', 'Goals For (GF)', 'Goals Against (GA)', '+ / -']

In [9]:
df = pd.DataFrame(rows, columns=columns)
df

,Team Name Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,0.388,292,278,14
2,Calgary Flames,1990,46,26,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,0.425,273,298,-25
...,...,...,...,...,...,...,...,...
577,Tampa Bay Lightning,2011,38,36,0.463,235,281,-46
578,Toronto Maple Leafs,2011,35,37,0.427,231,264,-33
579,Vancouver Canucks,2011,51,22,0.622,249,198,51
580,Washington Capitals,2011,42,32,0.512,222,230,-8
